In [1]:
import os
print(os.listdir('Medicaldataset'))

['Alovera', 'Hibiskus', 'Levender', 'Mint', 'Neem']


In [2]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
#preprocess.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical

# specifically for cnn
from tensorflow.keras.layers import Dropout, Flatten,Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [3]:
X=[]
Z=[]
IMG_SIZE=150
Alovera= f'Medicaldataset/Alovera'
Hibiskus=f'Medicaldataset/Hibiskus'
Levender=f'Medicaldataset/Levender'
Mint=f'Medicaldataset/Mint'
Neem =f'Medicaldataset/Neem'

In [4]:
def assign_label(img,plant_type):
    return plant_type

In [5]:
def make_train_data(plant_type,Medicaldataset):
    for img in tqdm(os.listdir(Medicaldataset)):
        label=assign_label(img,plant_type)
        path = os.path.join(Medicaldataset,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

In [6]:
make_train_data('Alovera',Alovera)
print(len(X))

100%|█████████████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 323.26it/s]

47


In [7]:
make_train_data('Hibiskus',Hibiskus)
print(len(X))

100%|█████████████████████████████████████████████████████████████████████████████████| 71/71 [00:00<00:00, 294.72it/s]

118


In [8]:
make_train_data('Mint',Mint)
print(len(X))

100%|█████████████████████████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 458.38it/s]

177


In [9]:
make_train_data('Neem',Neem)
print(len(X))

100%|█████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 193.26it/s]

218


In [10]:
make_train_data('Levender',Levender)
print(len(X))

100%|████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 1590.26it/s]

269


In [11]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255

In [12]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [13]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [14]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5, activation = "softmax"))

In [21]:
batch_size=24
epochs=80

from tensorflow.keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [22]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
     zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [23]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 96)        55392     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 96)        8

In [25]:
History = model.fit(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)
# model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,validation_data = (x_test,y_test))

Epoch 1/80
8/8 [==============================] - 4s 378ms/step - loss: 1.0391 - accuracy: 0.5593 - val_loss: 1.3077 - val_accuracy: 0.4559
Epoch 2/80
8/8 [==============================] - 3s 368ms/step - loss: 1.0446 - accuracy: 0.5424 - val_loss: 1.5289 - val_accuracy: 0.3676
Epoch 3/80
8/8 [==============================] - 3s 372ms/step - loss: 1.0189 - accuracy: 0.5198 - val_loss: 1.3165 - val_accuracy: 0.4706
Epoch 4/80
8/8 [==============================] - 3s 451ms/step - loss: 1.0013 - accuracy: 0.5537 - val_loss: 1.4987 - val_accuracy: 0.4412
Epoch 5/80
8/8 [==============================] - 6s 787ms/step - loss: 0.9002 - accuracy: 0.6441 - val_loss: 1.4632 - val_accuracy: 0.4559
Epoch 6/80
8/8 [==============================] - 6s 681ms/step - loss: 0.8383 - accuracy: 0.6441 - val_loss: 1.5459 - val_accuracy: 0.4412
Epoch 7/80
8/8 [==============================] - 5s 620ms/step - loss: 0.9328 - accuracy: 0.6328 - val_loss: 1.2348 - val_accuracy: 0.4853
Epoch 8/80
8/8 [====